# 02 - Tier 1 Benchmark (T4 GPU)

Run all Tier 1 models on the stratified sample set.
Designed for Colab Pro with T4 GPU.

**Runtime**: ~2 hours for all Tier 1 models on 100 pages

In [ ]:
# Install dependencies (Colab)
# !pip install -q -r requirements/colab_tier1.txt
# !pip install -e .

In [ ]:
import sys
import torch
from pathlib import Path

PROJECT_ROOT = Path(".").resolve().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")
print(f"VRAM: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB" if torch.cuda.is_available() else "")

In [ ]:
import yaml

with open(PROJECT_ROOT / "config" / "benchmark_config.yaml") as f:
    config = yaml.safe_load(f)

SAMPLE_SET = PROJECT_ROOT / "data" / "sample_sets" / "stratified_100.json"
IMAGE_DIR = PROJECT_ROOT / config["paths"]["image_dir"]
OUTPUT_DIR = PROJECT_ROOT / config["paths"]["raw_outputs_dir"]

# Tier 1 models in recommended order
TIER1_MODELS = [
    "doctr",
    "lighton_ocr",
    "got_ocr2",
    "florence2",
    "dots_ocr",
    "deepseek_ocr",
    "nanonets_ocr",
    "ocrflux",
    "granite_vision",
    "monkey_ocr",
]

In [ ]:
from src.pipeline.runner import run_model_on_sample_set, load_model_registry

registry = load_model_registry(str(PROJECT_ROOT / "config" / "model_registry.yaml"))
profiles = {}

for model_key in TIER1_MODELS:
    print(f"\n{'='*60}")
    print(f"Running: {model_key}")
    print(f"{'='*60}")
    try:
        profile = run_model_on_sample_set(
            model_key=model_key,
            sample_set_path=SAMPLE_SET,
            image_dir=IMAGE_DIR,
            output_dir=OUTPUT_DIR,
            registry=registry,
            device="cuda",
            skip_existing=True,
        )
        profiles[model_key] = profile
    except Exception as e:
        print(f"FAILED: {e}")
        import traceback
        traceback.print_exc()

In [ ]:
# Summary of all runs
import pandas as pd

rows = []
for model_key, profile in profiles.items():
    rows.append({
        "model": model_key,
        "pages": profile.total_pages,
        "avg_sec/page": round(profile.avg_time_per_page, 2),
        "pages/min": round(profile.pages_per_minute, 1),
        "peak_GPU_MB": round(profile.peak_gpu_memory_mb, 0),
    })

df = pd.DataFrame(rows)
print(df.to_string(index=False))